In [1]:
pip install pandas numpy statsmodels scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [6]:
# Set working directory (uncomment and modify the path as needed)
# import os
#os.chdir('')
#print(os.getcwd())

# Load the dataset
data = pd.read_csv('C:\\Users\\Admin\\Downloads\\Bootcamp Assignement\\NSSO68.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_23932\3439043763.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:\\Users\\Admin\\Downloads\\Bootcamp Assignement\\NSSO68.csv')


In [7]:
# Unique states
print(data['state_1'].unique())

['GUJ' 'ORI' 'CHTSD' 'MP' 'JRKD' 'WB' 'AP' 'MH' 'D&D' 'D&NH' 'MIZ' 'TRPR'
 'MANPR' 'ASSM' 'MEG' 'NAG' 'A&N' 'PNDCRY' 'TN' 'GOA' 'KA' 'KE' 'LKSDP'
 'SKM' 'Bhr' 'UP' 'RJ' 'ARP' 'DL' 'HR' 'Pun' 'HP' 'UT' 'Chandr' 'J$K']


In [8]:
# Subset data to state assigned
subset_data = data[data['state_1'] == 'GOA'][['foodtotal_q', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data.head())

       foodtotal_q  MPCE_MRP  MPCE_URP  Age  Meals_At_Home  \
22949    26.850600   8191.46   8381.25   56           58.0   
22950    35.625493   4674.52   4615.00   80           60.0   
22951    22.575320   2874.58   2575.50   42           60.0   
22952    25.967018   3240.10   3021.33   54           60.0   
22953    22.420334   2688.08   2790.00   28           60.0   

       Possess_ration_card  Education  No_of_Meals_per_day  
22949                  1.0       12.0                  2.0  
22950                  1.0       10.0                  2.0  
22951                  1.0       10.0                  2.0  
22952                  1.0       10.0                  2.0  
22953                  1.0       12.0                  2.0  


In [9]:
# Check for missing values
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Age'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(data['Education'].isna().sum())

0
0
0
0
7


In [10]:
# Impute missing values with mean
subset_data['Education'].fillna(subset_data['Education'].mean(), inplace=True)
print(subset_data['Education'].isna().sum())

0


In [18]:
# Drop any remaining rows with missing values in the selected columns
subset_data = subset_data.dropna(subset=['foodtotal_q', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education'])
print(subset_data.isna().sum())

foodtotal_q            0
MPCE_MRP               0
MPCE_URP               0
Age                    0
Meals_At_Home          0
Possess_ration_card    0
Education              0
No_of_Meals_per_day    0
dtype: int64


In [19]:
# Fit the regression model
model = ols('foodtotal_q ~ MPCE_MRP + MPCE_URP + Age + Meals_At_Home + Possess_ration_card + Education', data=subset_data).fit()

In [12]:
# Print the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            foodtotal_q   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.277
Method:                 Least Squares   F-statistic:                     29.31
Date:                Sun, 23 Jun 2024   Prob (F-statistic):           1.60e-29
Time:                        20:52:19   Log-Likelihood:                -1396.0
No. Observations:                 445   AIC:                             2806.
Df Residuals:                     438   BIC:                             2835.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               9.6802    

In [21]:
# Check for multicollinearity using Variance Inflation Factor (VIF)
X = subset_data[['MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education']]
X = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

               feature         VIF
0                const  213.483072
1             MPCE_MRP    2.893338
2             MPCE_URP    2.675995
3                  Age    1.338726
4        Meals_At_Home    1.171905
5  Possess_ration_card    1.096270
6            Education    1.376191


In [22]:
# Extract the coefficients from the model
coefficients = model.params

In [23]:
# Construct the equation
equation = f"y = {round(coefficients[0], 2)}"
for i in range(1, len(coefficients)):
    equation += f" + {round(coefficients[i], 6)}*x{i}"
print(equation)

y = 9.68 + 0.001879*x1 + -0.000122*x2 + 0.003818*x3 + 0.129623*x4 + -2.28727*x5 + 0.246891*x6


In [24]:
# Print the first values of selected columns
print(subset_data['MPCE_MRP'].head(1).values[0])
print(subset_data['MPCE_URP'].head(1).values[0])
print(subset_data['Age'].head(1).values[0])
print(subset_data['Meals_At_Home'].head(1).values[0])
print(subset_data['Possess_ration_card'].head(1).values[0])
print(subset_data['Education'].head(1).values[0])
print(subset_data['foodtotal_q'].head(1).values[0])

8191.46
8381.25
56
58.0
1.0
12.0
26.8506
